# 👗 From Noise to Fashion — Image Generation with Diffusion

Welcome! This notebook walks you through building and training an **conditional image generation model** on FashionMNIST: you can tell it which clothing category to generate, and it will create new images of that type from scratch.

Today you will build the core component of the "Graphic Designer Agent" from the full AI-run magazine application. From thin air, you will be able to generate images of clothes by using a model that learns to reconstruct a corrupted image.

This should feel almost like magic! Keep in mind, though, that due to time and resource constraints we will work on much simpler scenarios. The pipeline you build today is the foundation on which modern state-of-the-art image generation models are built. Those models — such as Stable Diffusion — use the very same approach, but are trained on billions of images across ~256 high-end GPUs for several weeks.

**Pipeline:**

1. 👗 **Load the data** — Set up FashionMNIST with 10 clothing classes
2. 💨 **Forward diffusion** — Mathematically destroy images by adding Gaussian noise
3. 🔬 **Visualize the noise** — Watch a clean image dissolve into static, step by step
4. 🧠 **Train a score model** — Teach a UNet to predict and reverse the noise
5. 🎨 **Generate new fashion** — Sample class-conditional images from pure noise
6. 🎞️ **Watch it denoise** — Visualize the full reverse diffusion journey

By the end, you'll be able to prompt the model for any of the 10 clothing categories — giving you a hands-on look at the core technology behind modern image generation systems.

---
## 0 📦 Imports

We import PyTorch for building and training the model, torchvision for the FashionMNIST dataset, and matplotlib for visualizations.

In [ ]:
import torch
import torch.nn.functional as F
from torch import optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import utils

# --- Set up device. If you don't have a GPU, consider using Google Colab ---
device = "cpu"
if torch.cuda.is_available(): #Use this if you have a CUDA-compatible GPU (e.g. Nvidida)
    device = "cuda"
elif torch.backends.mps.is_available(): #Use this if you have a Mac with M1/M2/M3/M4 chip
    device = "mps"
print(f"Using device: {device}")

In [ ]:
# cell to clone the repo for data and demo html on colab
!git clone -b week2-fashion-magazine-notebook https://github.com/eth-bmai-fs26/project.git
!git fetch && git checkout week2/fashion-magazine/notebook
%cd project/week2/notebook

---
## 1 👗 Load the Data

Before we can generate fashion images, we need to show the model what fashion looks like. **FashionMNIST** contains 60,000 grayscale training images across 10 clothing categories, each 28×28 pixels.

| Label | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
|-------|---|---|---|---|---|---|---|---|---|---|
| **Class** | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal | Shirt | Sneaker | Bag | Ankle boot |

We normalize each image to the range **[-1, 1]** (from raw [0, 255] pixel values). This matters for diffusion models: the noise process is defined over a zero-centered Gaussian, so keeping pixel values zero-centered keeps everything consistent.


In [ ]:
# --- Define transformations
# Transform to tensor and normalize dataset to [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # (mean, std) for single channel
])

# FashionMNIST class names (label index -> clothing category)
CLASS_NAMES = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]
NUM_CLASSES = len(CLASS_NAMES)

# Mapping from class names to indices (e.g., 'T-shirt/top' -> 0)
CLASS_TO_IDX = {name: idx for idx, name in enumerate(CLASS_NAMES)}

# --- Load FashionMNIST and create DataLoader (all 10 classes)
train_dataset = torchvision.datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

batch_size = 1024
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

# Let's look at one image to confirm
image, label = train_dataset[0]
print("Image shape:", image.shape)
print(f"Label: {label} ({CLASS_NAMES[label]})")
print(f"Pixel value range: {image.min()} to {image.max()}")

In [ ]:
utils.show_class_examples(train_dataset, CLASS_NAMES)

---
## 2 💨 Forward Diffusion

The key idea behind diffusion models is elegantly destructive: take a clean image and **gradually add Gaussian noise** over many timesteps until nothing recognizable remains. Then train a neural network to reverse the process.

The closed-form equation for the forward process lets us jump to *any* noise level in a single shot — no need to apply noise iteratively:

$$X_{i} = \sqrt{\bar{\alpha}_{i}} \, X_{0} + \sqrt{1-\bar{\alpha}_{i}} \, \varepsilon, \;\;\; \varepsilon \sim \mathcal{N}(0,I).$$

Here $\bar{\alpha}_i$ is the **cumulative product of the noise schedule** — it controls how much of the original image survives at timestep $i$:
- When $\bar{\alpha}_i \approx 1$ (early timesteps) — the image is nearly unchanged
- When $\bar{\alpha}_i \approx 0$ (late timesteps) — the image is almost pure Gaussian noise

> **Why is the closed form useful?** Training samples a random timestep $t$ for every image in the batch. Without this formula, computing $x_t$ would require applying the noise step $t$ times in a loop. With it, we get any noise level instantly — making training fast.

In [ ]:
# Hyperparameters for the diffusion process
TIMESTEPS = 800
beta_start = 0.0001
beta_end = 0.02

# Define the beta schedule (variance)
betas = torch.linspace(beta_start, beta_end, TIMESTEPS, device=device)

# Pre-calculate alphas and other values for the formula
alphas = 1. - betas
alphas_bar = torch.cumprod(alphas, dim=0) # Computes comulative product of alphas

sqrt_alphas_bar = torch.sqrt(alphas_bar)
sqrt_one_minus_alphas_bar = torch.sqrt(1. - alphas_bar)

# The Forward Noising Function
def add_noise(x_0, t):
    """
    Applies noise to an image x_0 at a specific timestep t.
    """
    # Sample random noise from a standard Gaussian
    noise = torch.randn_like(x_0)

    # Get the pre-calculated values for the given timesteps
    sqrt_alphas_bar_t = get_values_at_timestep(sqrt_alphas_bar, t)
    sqrt_one_minus_alphas_bar_t = get_values_at_timestep(sqrt_one_minus_alphas_bar, t)

    # Apply the formula to get the noisy image
    x_t = sqrt_alphas_bar_t * x_0 + sqrt_one_minus_alphas_bar_t * noise
    return x_t, noise

# Helper function to get the correct values for a batch of timesteps
def get_values_at_timestep(values, t):
    """
    Gathers the values from the `values` tensor at the indices specified by `t`.
    Reshapes the output to be compatible for broadcasting with image tensors.
    """
    batch_size = t.shape[0]
    # Gather values and reshape to (batch_size, 1, 1, 1) for broadcasting
    return values.gather(-1, t).reshape(batch_size, 1, 1, 1)

---
## 3 🔬 Visualize the Noising Process

Let's see the forward process in action. We'll take a single image and plot it at 10 evenly-spaced timesteps from $t=0$ (clean) to $t=T-1$ (pure noise).

**What to look for:**
- **Early timesteps** — The original image is clearly recognizable, just slightly grainier
- **Middle timesteps** — Fine details blur; shapes start to melt into the background
- **Late timesteps** — The image is unrecognizable — pure Gaussian noise

This progression is exactly what the model has to *undo* during generation.

In [ ]:
# Visualization of the forward pass
image, _ = train_dataset[0] # Get first image
image = image.to(device).unsqueeze(0) # Add batch dimension and move to device

num_steps = 10  # Number of noise levels to visualize
vis_timesteps = torch.linspace(0, TIMESTEPS - 1, num_steps).long().tolist()
noisy_images = []
titles = []

# Generate noisy images at specified timesteps
for timestep in vis_timesteps:
    noisy_img, _ = add_noise(image, torch.tensor([timestep], device=device))
    noisy_images.append(noisy_img)
    titles.append(f"t = {timestep}")

print("Visualizing the forward noising process:")
utils.show_images(torch.cat(noisy_images, dim=0), titles=titles)

---
## 4 🧠 Reverse Diffusion — Training the Denoiser

Now for the clever part. We've seen how to *destroy* an image. The question is: can a neural network learn to *reverse* this process?

The answer is yes — by training a **UNet** to predict the noise $\varepsilon$ that was added at each timestep. If the model can accurately estimate the noise, it can subtract it to recover a slightly cleaner image. Repeat 800 times starting from pure static, and a coherent image emerges.

The model is **class-conditional**: we pass the clothing category label alongside the noisy image, so the model learns to reverse noise in a class-aware way. This is how you steer generation toward a specific category at sampling time.

### 4.1 🏋️ Train the UNet

We import `SimpleUNet` — a lightweight UNet with skip connections that takes a noisy image $x_t$, the timestep $t$, and the class label as inputs, and outputs a **predicted noise tensor of the same shape**.

The training objective is beautifully simple: minimize the MSE between the predicted noise and the actual noise that was added. This is the core of the DDPM training algorithm.

| Input | What it is |
|-------|-----------|
| `x_t` | The noisy image at timestep $t$ |
| `t` | The timestep — tells the model how noisy the input is |
| `labels` | The class label — steers denoising toward a specific category |

In [ ]:
from unet import SimpleUNet

# --- Initialize the model which we use to predict the score function.
score_model = SimpleUNet(base_ch=64, emb_dim=64, num_classes=NUM_CLASSES).to(device) #Set base_ch to 16 if this takes too long on your laptop
score_model.train()
optimizer = optim.AdamW(score_model.parameters(), lr=1e-4)

# --- Training loop, learning to reverse the noise. ---
epochs = 20

for epoch in range(1, epochs + 1):
    for x_0, labels in train_loader: # CHANGE 2: unpack labels alongside images
        optimizer.zero_grad(set_to_none=True)
        x_0 = x_0.to(device)
        labels = labels.to(device)  # CHANGE 2: move labels to device
        batch_dimension = x_0.size(0)
        t = torch.randint(0, TIMESTEPS, (batch_dimension,), device=device).long() # choose random timestep for every sample from the input

        # Add noise using the forward process
        x_t, noise = add_noise(x_0, t)

        # Predict the added noise, conditioned on the class label
        predicted_noise = score_model(x_t, t, labels)  # CHANGE 2: pass labels to model

        # Learn to reverse the noise
        loss = F.mse_loss(predicted_noise, noise)

        # Take an optimization step
        loss.backward()
        optimizer.step()

    if epoch%1 == 0:
        print(f"Finished epoch {epoch}. Current loss: loss={loss.item():.4f}")

print("Training is finished!")

### 4.2 🔁 The Sampling Equations

Training taught the model to estimate noise. Now we use that estimate to iteratively denoise, stepping backwards from $t=T$ to $t=0$.

At each step, we compute the **posterior mean** — our best guess for a slightly-less-noisy image:

$$\mu(x_i) = \frac{1}{\sqrt{\alpha_i}} \left( x_i - \frac{1-\alpha_i}{\sqrt{1 - \bar{\alpha}_{i}}} \bar{\varepsilon}_{i} \right)$$

We also inject a small amount of random noise scaled by the **posterior variance**, to maintain sample diversity:

$$\sigma^{2}(i) = \frac{(1 - \alpha_{i}) (1 - \bar{\alpha}_{i-1})}{1 - \bar{\alpha}_{i}}$$

Putting it together, the full reverse step is:

$$x_{i-1} = \frac{1}{\sqrt{\alpha_i}} \left( x_i - \frac{1-\alpha_i}{\sqrt{1 - \bar{\alpha}_{i}}} \bar{\varepsilon}_{i} \right) + \sigma(i) z$$

> **Why add noise during sampling?** Without the $\sigma(i) z$ term, every run from the same starting noise would produce the identical image. The injected stochasticity is what gives diffusion models their rich, diverse outputs.

---
## 5 🎨 Generate New Fashion

Training is done — now comes the fun part. We sample new images by starting from **pure Gaussian noise** and running the reverse diffusion process 800 steps, guided by the trained UNet.

Because the model is class-conditional, we can ask it to generate any of the 10 clothing categories. Change `class_label` to any number from 0 to 9 and watch the model invent a new garment from static.

In [8]:
# Precompute helpful buffers for sampling
alphas_bar_previous = torch.cat([torch.tensor([1.0], device=device), alphas_bar[:-1]], dim=0)
sqrt_alphas_inverse = torch.sqrt(1.0 / alphas)
inverse_sqrt_one_minus_alphas_bar = torch.sqrt(1.0 - alphas_bar)

# Posterior mean coefficient
coef_eps = (1 - alphas) / inverse_sqrt_one_minus_alphas_bar

# Posterior variance
posterior_variance = (1 - alphas) * (1 - alphas_bar_previous) / (1 - alphas_bar)
posterior_log_variance_clipped = torch.log(torch.clamp(posterior_variance, min=1e-20))

In [ ]:
@torch.no_grad()
def sample(score_model, num_samples=16, class_label=0):
    score_model.eval()
    # Start from random noise
    x_t = torch.randn(num_samples, 1, 28, 28, device=device)

    # Class label tensor — same label for all samples in this batch
    labels = torch.full((num_samples,), class_label, device=device, dtype=torch.long)

    # Now we reverse the noising
    for timestep in reversed(range(TIMESTEPS)):
        t = torch.full((num_samples,), timestep, device=device, dtype=torch.long)

        # Denoising, conditioned on the class label
        mean = (x_t - coef_eps[timestep] * score_model(x_t, t, labels)) * sqrt_alphas_inverse[timestep]

        if timestep > 0:
            #Add noise for sample diversity
            noise = torch.randn_like(x_t)
            var = posterior_variance[timestep]
            x_t = mean + torch.sqrt(var) * noise
        else:
            x_t = mean
    return x_t

In [ ]:
num_samples = 16
class_label = 0 # tshirt

# Generate samples conditioned on the chosen class
samples = sample(score_model, num_samples=num_samples, class_label=class_label)

# unnormalize back to [0,1] for display
samples = (samples.clamp(-1, 1) + 1) / 2.0

print(f"16 samples of class {class_label}: '{CLASS_NAMES[class_label]}'.")
utils.show_images(samples[:num_samples].cpu(), titles=[CLASS_NAMES[class_label] for _ in range(num_samples)], grid=True)

---
## 6 🎞️ Watch It Denoise

A single image, visualized at 10 checkpoints across the full denoising journey — from pure noise all the way to the finished garment.

**What to look for:**
- **High $t$** — Indistinguishable static; the model is just getting started
- **Mid $t$** — Rough shapes and outlines begin to emerge
- **Low $t$** — Fine texture and detail snap into place in the final steps

This is the reverse of Section 3 — the same 10-step window, but running backwards.

In [ ]:
@torch.no_grad()
def denoising_process_single_sample(
    score_model,
    class_label: int = 0,                         # which class to generate
    record_timesteps: list[int] | None = None,    # which t to keep (e.g., [TIMESTEPS-1,...,0]); None = keep all
):
    score_model.eval()

    x_t = torch.randn(1, 1, 28, 28, device=device)
    start_t = TIMESTEPS - 1

    # Class label tensor (batch size 1)
    labels = torch.tensor([class_label], device=device, dtype=torch.long)

    record_set = set(record_timesteps) if record_timesteps is not None else None
    trace = []

    # Optionally record the starting x_T
    if record_set is None or start_t in record_set:
        trace.append((start_t, x_t.detach().clone()))

    for timestep in reversed(range(TIMESTEPS)):
        t = torch.full((1,), timestep, device=device, dtype=torch.long)

        # mean term: (x - coef_eps_i * eps_theta) / sqrt(alpha_i), conditioned on class
        mean = (x_t - coef_eps[timestep] * score_model(x_t, t, labels)) * sqrt_alphas_inverse[timestep]

        if timestep > 0:
            noise = torch.randn_like(x_t)
            var = posterior_variance[timestep]
            x_t = mean + torch.sqrt(var) * noise
        else:
            x_t = mean  # final step has no noise

        if record_set is None or timestep in record_set:
            trace.append((timestep, x_t.detach().clone()))

    # trace is in descending t order already (T-1 -> 0)
    return x_t, trace


In [ ]:

# ---- Example usage: record specific timesteps to mirror your forward viz ----
vis_timesteps = torch.linspace(TIMESTEPS - 1, 0, 10).long().tolist()
class_label = 2

# Run reverse sampling for one sample and keep only those steps
x0, trace = denoising_process_single_sample(score_model, class_label=class_label, record_timesteps=vis_timesteps)

# Prepare images (ordered by vis_timesteps for consistent titles)
# trace is a list of (t, x_t); convert to dict for quick indexing
trace_dict = {t: x_t for (t, x_t) in trace}
imgs = []
titles = []
for t_val in vis_timesteps:
    x_t = trace_dict[t_val]
    # unnormalize from [-1,1] to [0,1] for display
    x_disp = (x_t.clamp(-1, 1) + 1) / 2.0
    imgs.append(x_disp.squeeze(0))  # remove batch dim -> (1,28,28)
    titles.append(f"t = {t_val}")

# Visualize the reverse denoising process
print(f"Visualizing the reverse denoising process for class {class_label}: '{CLASS_NAMES[class_label]}':")
utils.show_images(torch.stack(imgs).cpu(), titles=titles)